In [1]:
import numpy as np
import pandas as pd

import gc
import os

from tqdm import tqdm_notebook, tqdm
import time

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score, mean_squared_error

import matplotlib.pylab as plt
%matplotlib inline

from Helpers import *

# Бенчмарки
В данной работе проводится сравнение работы Vowpall Wabbit и нашей FM на основе датасета MovieLens
Будут проверены следующие модификации:
- **movielens - 1m**  с нормализацией признаков
- **movielens - 1m**  без нормализации признаков
- **movielens - 10m** без нормализации признаков
- **movielens - 10m** как задача классификации
- **movielens - 20m** без нормализации признаков


# Preprocessing
### Предподготовка датасета в pandas

**Основные паки признаков:**
- OHE юзеров и фильмов
- Время (число месяцев, прошедшее от первой записи)
- OHE жанров фильмов
- Признаки пользователей (только для ml-1m)
- Факт оценивания фильма (OHE)

# Спойлер
*Здесь будет выжимка результатов лучших алгоритмов*

Отмечу, что некоторые модификации пропускались, если не вышло получить на них внятного хорошего результата

In [30]:
best_models = pd.read_csv('best_models.csv').dropna()
best_models

,Dataset,Model,RMSE,Time (sec)
0,ml-1m-norm,FM – SGD (5 passes),1.186,121
1,ml-1m-norm,Pairwise FM – SGD (5 passes),1.186,255
2,ml-1m-norm,FM – ALS (5 passes),0.911,165
3,ml-1m-norm,Pairwise FM – ALS (5 passes),0.912,859
4,ml-1m-norm,VW SGD (10 passes),1.149,10
5,ml-1m-norm,VW SGD (80 passes),1.122,62
6,ml-1m-norm,VW (5 passes),1.068,6
7,ml-1m-norm,VW (80 passes),0.944,72
8,ml-1m,FM – SGD (5 passes),1.006,113
9,ml-1m,Pairwise FM – SGD (5 passes),1.020,214


In [31]:
pd.pivot_table(best_models, values='RMSE', index='Model', columns='Dataset')

Dataset,ml-10m,ml-1m,ml-1m-norm,ml-20m
Model,,,,
"FM – ALS (1 passes, HT)",NaN,NaN,NaN,0.937
"FM – ALS (10 passes, HT)",0.945,NaN,NaN,NaN
FM – ALS (2 passes),NaN,0.911,NaN,NaN
FM – ALS (5 passes),0.889,NaN,0.911,NaN
"FM – ALS (5 passes, HT)",0.948,NaN,NaN,NaN
FM – SGD (10 passes),NaN,1.029,NaN,NaN
FM – SGD (5 passes),1.051,1.006,1.186,1.012
Pairwise FM – ALS (5 passes),0.852,0.889,0.912,NaN
"Pairwise FM – ALS (5 passes, HT)",0.919,NaN,NaN,NaN


# Regression

Попробуем для начала на датасете ml-1m, используя нормализацию и все полученные фичи, кроме ratedMovies

Чуть позже же решим, делает ли нам нормализация категориальных признаков хорошо или нет

In [8]:
folder = 'SourceData/ml-1m/'
test_size = 0.25
train, test = build_movielens(folder, test_size)
field_info = get_offset_stats(train, test)
train.head()

load ratings....
calculation of monthes....
load movies....
build genres ohe....
join dataframes....
load users info....
train/test split...
building rated movies history (on train)....
preprocessing done....


,user,movie,rating,monthes,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,...,genre_15,genre_16,genre_17,genre_18,genre_19,gender,age,occupation,zip,ratedMovies
1,2,1193,5,0.216187,0,0,0,0,0,0,...,0,0,0,0,0,1,1.000000,16,2248,"[1193, 3105, 2321, 1962, 1207, 2028, 1246, 306..."
2,12,1193,4,0.216187,0,0,0,0,0,0,...,0,0,0,0,0,1,0.446533,12,1165,"[1193, 2804, 1198, 593, 1247, 1641, 1221, 111,..."
3,15,1193,4,0.216187,0,0,0,0,0,0,...,0,0,0,0,0,1,0.446533,7,904,"[1193, 3408, 3105, 2321, 527, 2762, 260, 2028,..."
4,17,1193,5,0.216187,0,0,0,0,0,0,...,0,0,0,0,0,1,0.893066,1,3187,"[1193, 595, 2321, 720, 1270, 527, 1097, 2762, ..."
5,18,1193,4,0.216187,0,0,0,0,0,0,...,0,0,0,0,0,0,0.321533,3,3227,"[1193, 1197, 919, 595, 2018, 2797, 527, 48, 10..."


In [8]:
train2format(train, fm_extractor, field_info, 'Generated/train_1m_norm_user_reg.fm',
             with_normalization=True, with_user_features=True)
test2format(test, fm_extractor, field_info, 'Generated/test_1m_norm_user_reg.fm', 'Generated/ytest_1m_norm_user_reg', 
            with_normalization=True, with_user_features=True)

100%|██████████| 250052/250052 [02:41<00:00, 1551.64it/s]


### Для начала посмотрим, как же пользоваться этой нашей тулзой

In [51]:
!./FM --help

Library for using factorization algorithm
Usage:
  Factorization machines [OPTION...]

  -l, --learning_rate arg       Learning rate value, default 0.1
      --bias                    Use bias or not
      --linear                  Use linear or not
      --pairwise arg            Two-way interactions order
  -r, --regularization_const arg
                                Regularization constant, default 0
  -i, --iterations arg          Number of iterations, default 100
  -m, --learning_method arg     Learning method (SGD, ALS), default SGD
  -g, --inplace arg             Storage (inplace, memory), default memory
  -t, --train_filename arg      Training file name
  -e, --test_filename arg       Testing file name
  -s, --task_type arg           Task type parameter
      --hash_size arg           Positive hash size if use hashing trick, else
                                -1, default -1
      --hash_random_seed arg    Random seed of hashing
  -h, --help                    Usage descript

### Жадный поиск гиперпараметров

Начнем жадно подбирать параметры для SGD и ALD независимо. Начнем со старого доброго **SGD**

In [23]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_norm_user_reg.fm -e Generated/test_1m_norm_user_reg.fm \
    --learning_rate 0.0001 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.24986 Test=1.25241
iter=1 Train=1.24717 Test=1.24976
iter=2 Train=1.24731 Test=1.24994
iter=3 Train=1.2479 Test=1.25057
iter=4 Train=1.24863 Test=1.25135
Time:		215 seconds


In [20]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_norm_user_reg.fm -e Generated/test_1m_norm_user_reg.fm \
    --learning_rate 0.00001 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.2283 Test=1.23035
iter=1 Train=1.19918 Test=1.20147
iter=2 Train=1.19037 Test=1.19275
iter=3 Train=1.18601 Test=1.18845
iter=4 Train=1.1833 Test=1.18577
Time:		255 seconds


In [11]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_norm_user_reg.fm -e Generated/test_1m_norm_user_reg.fm \
    --learning_rate 0.00001 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.22793 Test=1.22998
iter=1 Train=1.19955 Test=1.20183
iter=2 Train=1.19078 Test=1.19316
iter=3 Train=1.18639 Test=1.18882
iter=4 Train=1.18362 Test=1.18608
Time:		121 seconds


In [29]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_norm_user_reg.fm -e Generated/test_1m_norm_user_reg.fm \
    --learning_rate 0.000001 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.57427 Test=1.57586
iter=1 Train=1.29442 Test=1.29592
iter=2 Train=1.24919 Test=1.25068
iter=3 Train=1.22833 Test=1.22986
iter=4 Train=1.21293 Test=1.21453
Time:		127 seconds


In [26]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_norm_user_reg.fm -e Generated/test_1m_norm_user_reg.fm \
    --learning_rate 0.00001 --regularization_const 0.01 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.23412 Test=1.23618
iter=1 Train=1.20378 Test=1.20607
iter=2 Train=1.19402 Test=1.19641
iter=3 Train=1.18942 Test=1.19184
iter=4 Train=1.18681 Test=1.18926
Time:		182 seconds


**Лучший SGD набирает порядка 1.18 RMSE на отложенной выборке**

### ALS

In [24]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_norm_user_reg.fm -e Generated/test_1m_norm_user_reg.fm \
    --learning_method ALS --hash_size -1 -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=0.883127 Test=0.925434
iter=1 Train=0.863028 Test=0.923017
iter=2 Train=0.849879 Test=0.919942
iter=3 Train=0.840508 Test=0.915377
iter=4 Train=0.834058 Test=0.911848
Time:		859 seconds


In [28]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_norm_user_reg.fm -e Generated/test_1m_norm_user_reg.fm \
    --learning_method ALS --hash_size -1 -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=0.904478 Test=0.920831
iter=1 Train=0.895432 Test=0.911426
iter=2 Train=0.894925 Test=0.910885
iter=3 Train=0.894861 Test=0.910814
iter=4 Train=0.894837 Test=0.910785
Time:		165 seconds


In [29]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_norm_user_reg.fm -e Generated/test_1m_norm_user_reg.fm \
    --learning_method ALS --regularization_const 0.00001 --hash_size -1 -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=0.937126 Test=0.94576
iter=1 Train=0.935544 Test=0.943866
iter=2 Train=0.93519 Test=0.943468
iter=3 Train=0.934827 Test=0.943097
iter=4 Train=0.934496 Test=0.942765
Time:		161 seconds


**Лучший скор - 0.910**

**ALS результаты и вовсе внушительные, причем наличие парных взаимодействий на данном датасете на эффективность особо не влияет**

## VW
### Приведем наши датасеты к формату VW

In [35]:
fm2vw('Generated/train_1m_norm_user_reg.fm', 'Generated/train_1m_norm_user_reg.vw')
fm2vw('Generated/test_1m_norm_user_reg.fm', 'Generated/test_1m_norm_user_reg.vw')

750157it [00:01, 673301.96it/s]
250052it [00:00, 718377.71it/s]


### И снова как мы любим начнем жадно выбирать параметры

Начнем с SGD и найдем хороший learning rate

In [39]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.01 -b 14 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 4 seconds
Start evaluation
Validation RMSE:	 1.198904507129869
Time:		6 seconds


In [37]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 14 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 4 seconds
Start evaluation
Validation RMSE:	 1.1675452150871288
Time:		6 seconds


In [40]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.0001 -b 14 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 4 seconds
Start evaluation
Validation RMSE:	 1.5458927830958853
Time:		6 seconds


### Число проходов

Для лучшей конфигуразции будем увеличивать число проходов

In [38]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 14 --passes 10 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 8 seconds
Start evaluation
Validation RMSE:	 1.1483929069769112
Time:		10 seconds


In [43]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 14 --passes 20 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 15 seconds
Start evaluation
Validation RMSE:	 1.1354925339260975
Time:		18 seconds


In [44]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 14 --passes 40 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 31 seconds
Start evaluation
Validation RMSE:	 1.1269815304667365
Time:		33 seconds


In [55]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 14 --passes 80 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 62 seconds
Start evaluation
Validation RMSE:	 1.1224261716600354
Time:		64 seconds


**Лучший результат на VW SGD - 1.1224**

Попробуем на хитрых дефолтных параметрах

In [46]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.001 -b 14 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 4 seconds
Start evaluation
Validation RMSE:	 1.397999386906915
Time:		6 seconds


In [48]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.01 -b 14 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 4 seconds
Start evaluation
Validation RMSE:	 1.068677025561811
Time:		6 seconds


In [56]:
t_start = time.time()
!vw Generated/train_1m_norm_user_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.01 -b 14 --passes 80 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_norm_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_norm_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 70 seconds
Start evaluation
Validation RMSE:	 0.9439532525754946
Time:		72 seconds


**Лучший результат на VW - 0.944**

Напомню, что лучший скор нашей тулзы был 0.910

**И хотя какой-то хороший бейзлайн на VW получается довольно шустро, такого же качества, которое давал нам ALS добиться на VW не удалось**


# Ненормированные ml-1m
Посмотрим как себя поведут утилиты на ненормированных признаках

In [9]:
train2format(train, fm_extractor, field_info, 'Generated/train_1m_user_reg.fm',
             with_normalization=False, with_user_features=True)
test2format(test, fm_extractor, field_info, 'Generated/test_1m_user_reg.fm', 'Generated/ytest_1m_user_reg', 
            with_normalization=False, with_user_features=True)

100%|██████████| 250052/250052 [01:51<00:00, 2241.35it/s]


In [10]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_rate 0.01 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.16565 Test=1.17666
iter=1 Train=1.12185 Test=1.13626
iter=2 Train=1.10682 Test=1.12384
iter=3 Train=1.10884 Test=1.12796
iter=4 Train=1.11811 Test=1.13904
Time:		215 seconds


### Добавим регуляризацию

Как видим есть риск сильно перестараться

In [18]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm  \
    --learning_rate 0.01 --learning_method SGD --regularization_const 0.1 --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.81831 Test=1.81998
iter=1 Train=1.80246 Test=1.80387
iter=2 Train=1.80235 Test=1.80342
iter=3 Train=1.80293 Test=1.80416
iter=4 Train=1.80428 Test=1.80517
Time:		213 seconds


In [20]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_rate 0.01 --learning_method SGD --regularization_const 0.01 --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.07767 Test=1.08947
iter=1 Train=1.01272 Test=1.02826
iter=2 Train=0.987602 Test=1.00637
iter=3 Train=0.986864 Test=1.00808
iter=4 Train=0.997491 Test=1.02045
Time:		214 seconds


In [9]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_rate 0.01 --learning_method SGD --regularization_const 0.01 --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.08072 Test=1.09072
iter=1 Train=1.02721 Test=1.03871
iter=2 Train=1.00472 Test=1.01701
iter=3 Train=0.99589 Test=1.00862
iter=4 Train=0.993802 Test=1.00679
Time:		113 seconds


In [19]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_rate 0.01 --learning_method SGD --regularization_const 0.001 --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.15296 Test=1.16405
iter=1 Train=1.10143 Test=1.11605
iter=2 Train=1.07779 Test=1.09532
iter=3 Train=1.07129 Test=1.09123
iter=4 Train=1.07314 Test=1.09513
Time:		219 seconds


### Добавим итераций лучшим алгоритмам


In [23]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_rate 0.002 --learning_method SGD --regularization_const 0.01 --hash_size -1 -g inplace -i 10
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.19262 Test=1.19966
iter=1 Train=1.13433 Test=1.14319
iter=2 Train=1.09052 Test=1.10094
iter=3 Train=1.05602 Test=1.06792
iter=4 Train=1.02983 Test=1.04315
iter=5 Train=1.01051 Test=1.02516
iter=6 Train=0.996751 Test=1.01261
iter=7 Train=0.987356 Test=1.00432
iter=8 Train=0.981408 Test=0.999345
iter=9 Train=0.978387 Test=0.997183
Time:		389 seconds


In [10]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_rate 0.01 --learning_method SGD --regularization_const 0.01 --hash_size -1 -g inplace -i 10
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=1.08072 Test=1.09072
iter=1 Train=1.02721 Test=1.03871
iter=2 Train=1.00472 Test=1.01701
iter=3 Train=0.99589 Test=1.00862
iter=4 Train=0.993802 Test=1.00679
iter=5 Train=0.995354 Test=1.00846
iter=6 Train=0.99901 Test=1.01218
iter=7 Train=1.00396 Test=1.01715
iter=8 Train=1.00975 Test=1.02291
iter=9 Train=1.01606 Test=1.0292
Time:		230 seconds


**Лучший результат нашей тулзы на SGD показал 0.997 RMSE на отложенной выборке**
### ALS

In [49]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_method ALS --hash_size -1 -g memory -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=0.864562 Test=0.917794
iter=1 Train=0.836503 Test=0.904523
iter=2 Train=0.822554 Test=0.896032
iter=3 Train=0.815551 Test=0.891903
iter=4 Train=0.811589 Test=0.889514
Time:		781 seconds


In [51]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_method ALS --hash_size -1 -g memory -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=0.904478 Test=0.920831
iter=1 Train=0.895432 Test=0.911426
iter=2 Train=0.894925 Test=0.910885
iter=3 Train=0.894861 Test=0.910814
iter=4 Train=0.894837 Test=0.910785
Time:		164 seconds


**Обращаю внимание какой хороший эффект дают парные фичи, но при этом сильно замедляя работу**

In [52]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_method ALS --regularization_const 0.01 --hash_size -1 -g memory -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=0.904478 Test=0.920826
iter=1 Train=0.895433 Test=0.911422
iter=2 Train=0.894925 Test=0.910881
iter=3 Train=0.894861 Test=0.910809
iter=4 Train=0.894837 Test=0.91078
Time:		157 seconds


In [57]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_method ALS --regularization_const 0.001 --hash_size -1 -g memory -i 2
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=0.904478 Test=0.92083
iter=1 Train=0.895432 Test=0.911426
Time:		75 seconds


In [56]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_1m_user_reg.fm -e Generated/test_1m_user_reg.fm \
    --learning_method ALS --hash_size -1 -g memory -i 10
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 750157 rows
Target from 1 to 5
Max feature index is 13476
Preprocessing
Processed 250052 rows
Target from 1 to 5
Max feature index is 13476
iter=0 Train=0.864562 Test=0.917794
iter=1 Train=0.836503 Test=0.904523
iter=2 Train=0.822554 Test=0.896032
iter=3 Train=0.815551 Test=0.891903
iter=4 Train=0.811589 Test=0.889514
iter=5 Train=0.809086 Test=0.88802
iter=6 Train=0.807374 Test=0.887033
iter=7 Train=0.806132 Test=0.886277
iter=8 Train=0.805194 Test=0.885726
iter=9 Train=0.804467 Test=0.885303
Time:		1844 seconds


**Итого лучший ALS с 10 проходами и 4 компонентами на парные взаимодействия показал 0.885 RMSE, обучаясь порядка 30 минут**

**Результат 0.91 же мог быть достигнут и без парных взаимодействий за 2 прохода достаточно шустро**

## Vowpal wabbit

In [30]:
fm2vw('Generated/train_1m_user_reg.fm', 'Generated/train_1m_user_reg.vw')
fm2vw('Generated/test_1m_user_reg.fm', 'Generated/test_1m_user_reg.vw')

750157it [00:00, 915839.16it/s]
250052it [00:00, 799685.63it/s]


In [39]:
t_start = time.time()
!vw Generated/train_1m_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 14 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 4 seconds
Start evaluation
Validation RMSE:	 1.1573913484777147
Time:		6 seconds


In [40]:
t_start = time.time()
!vw Generated/train_1m_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 14 --passes 10 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 8 seconds
Start evaluation
Validation RMSE:	 1.139532181507997
Time:		10 seconds


In [43]:
t_start = time.time()
!vw Generated/train_1m_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 14 --passes 80 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 69 seconds
Start evaluation
Validation RMSE:	 1.1148919874990038
Time:		71 seconds


In [46]:
t_start = time.time()
!vw Generated/train_1m_user_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.002 -b 14 --passes 80 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_1m_user_reg.vw -r VwHelpFiles/pred --quiet
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_1m_user_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 43 seconds
Start evaluation
Validation RMSE:	 1.109155811552204
Time:		45 seconds


**Лучший VW дал 1.109, что серьезно уступает SGD режиму FM**

**ALS режим же на данном датасете вообще из другой лиги**



# ml-10M100K

Сегодня мы многое поняли - например то, что нормировка не бог весть какая хорошая идея.

Так что в дальнейшем будем рассматривать ненормированные датасеты.

Более того, информация о пользователям нам уже не дана, так что эти фичи исключаются тоже

In [7]:
folder = 'SourceData/ml-10M100K/'
test_size = 0.25
train, test = build_movielens(
    folder, test_size, with_genres=True,
    with_users_info=False, with_rated_movies=False)
train.head()

load ratings....
calculation of monthes....
load movies....
build genres ohe....
join dataframes....
users info skipped
train/test split...
rated movies history skipped
preprocessing done....


,user,movie,rating,monthes,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,...,genre_14,genre_15,genre_16,genre_17,genre_18,genre_19,genre_20,genre_21,genre_22,genre_23
1,139,122,3,0.417480,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,215,122,4,0.708984,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
5,217,122,3,0.120850,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
6,281,122,3,0.120850,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
7,326,122,3,0.109863,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [34]:
field_info = get_offset_stats(train, test)
train2format(train, fm_extractor, field_info, 'Generated/train_10m_reg.fm',)
test2format(
    test, fm_extractor, field_info,
    'Generated/test_10m_reg.fm',
    'Generated/ytest_10m_reg'
)
!head -n 3 Generated/train_10m_reg.fm

100%|██████████| 2500013/2500013 [12:24<00:00, 3357.47it/s]


3 139:1 71690:1 136702:0.4 136707:1 136722:1 
4 215:1 71690:1 136702:0.7 136707:1 136722:1 
3 217:1 71690:1 136702:0.1 136707:1 136722:1 


### Найдем порядок хорошего learning rate

In [36]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
    --learning_rate 0.01 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 136726
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 136726
iter=0 Train=1.14542 Test=1.14962
iter=1 Train=1.09418 Test=1.10123
iter=2 Train=1.07929 Test=1.08916
iter=3 Train=1.07324 Test=1.08581
iter=4 Train=1.07011 Test=1.08503
Time:		1230 seconds


In [37]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
    --learning_rate 0.001 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 136726
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 136726
iter=0 Train=1.15223 Test=1.15402
iter=1 Train=1.11143 Test=1.11475
iter=2 Train=1.09046 Test=1.09506
iter=3 Train=1.07533 Test=1.08097
iter=4 Train=1.06291 Test=1.06942
Time:		1314 seconds


In [42]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
    --learning_rate 0.003 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 136726
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 136726
iter=0 Train=1.1351 Test=1.13817
iter=1 Train=1.10296 Test=1.10807
iter=2 Train=1.086 Test=1.09274
iter=3 Train=1.07706 Test=1.08515
iter=4 Train=1.07759 Test=1.08684
Time:		1151 seconds


### Попробуем с регуляризацией

In [38]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
    --learning_rate 0.001 --regularization_const 0.01 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 136726
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 136726
iter=0 Train=1.15646 Test=1.15784
iter=1 Train=1.094 Test=1.09629
iter=2 Train=1.06581 Test=1.06888
iter=3 Train=1.05545 Test=1.05912
iter=4 Train=1.05259 Test=1.05676
Time:		1147 seconds


### Без парных взаимодействий

In [15]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
    --learning_rate 0.001 --regularization_const 0.01 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 136726
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 136726
iter=0 Train=1.16671 Test=1.16795
iter=1 Train=1.10823 Test=1.11014
iter=2 Train=1.07654 Test=1.079
iter=3 Train=1.05848 Test=1.06141
iter=4 Train=1.04788 Test=1.05121
Time:		719 seconds


### И с хэшинг триком

Регуляризацию стало возможным ослабить, ведь и сам прием имеет некоторый такой эффект

In [43]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
    --learning_rate 0.001 --regularization_const 0.01 --learning_method SGD --hash_size 32000 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 31999
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 31999
iter=0 Train=1.20627 Test=1.2074
iter=1 Train=1.18537 Test=1.18677
iter=2 Train=1.18037 Test=1.18194
iter=3 Train=1.18114 Test=1.1829
iter=4 Train=1.18496 Test=1.18696
Time:		1418 seconds


In [45]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
    --learning_rate 0.0001 --regularization_const 0.0001 --learning_method SGD --hash_size 32000 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 31999
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 31999
iter=0 Train=1.21645 Test=1.21701
iter=1 Train=1.19453 Test=1.1952
iter=2 Train=1.17971 Test=1.18049
iter=3 Train=1.16881 Test=1.16969
iter=4 Train=1.16034 Test=1.16132
Time:		1406 seconds


### ALS

На ALS с честными фичами без хэшинг трика запускать мне страшно

Так что будем постепенно увеличивать число значений хэш функции 

In [46]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
   --pairwise 0 --regularization_const 0.0001 --learning_method ALS --hash_size 16000 -g memory -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 15999
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 15999
iter=0 Train=1.00607 Test=1.0069
iter=1 Train=1.00037 Test=1.00121
iter=2 Train=0.998275 Test=0.999098
iter=3 Train=0.996887 Test=0.9977
iter=4 Train=0.99592 Test=0.996725
Time:		791 seconds


In [8]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
   --pairwise 0 --regularization_const 0.0001 --learning_method ALS --hash_size 32000 -g memory -i 10
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 31999
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 31999
iter=0 Train=0.980275 Test=0.981755
iter=1 Train=0.974717 Test=0.976171
iter=2 Train=0.972991 Test=0.974422
iter=3 Train=0.971827 Test=0.97324
iter=4 Train=0.970968 Test=0.972368
iter=5 Train=0.970318 Test=0.971708
iter=6 Train=0.969821 Test=0.971202
iter=7 Train=0.969439 Test=0.970813
iter=8 Train=0.969144 Test=0.970513
iter=9 Train=0.968916 Test=0.97028
Time:		1596 seconds


In [9]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
   --pairwise 0 --regularization_const 0.0001 --learning_method ALS --hash_size 64000 -g memory -i 10
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 63999
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 63999
iter=0 Train=0.953901 Test=0.957139
iter=1 Train=0.947997 Test=0.951252
iter=2 Train=0.946475 Test=0.949724
iter=3 Train=0.94545 Test=0.94869
iter=4 Train=0.944681 Test=0.947911
iter=5 Train=0.944073 Test=0.947293
iter=6 Train=0.94358 Test=0.946791
iter=7 Train=0.943175 Test=0.946378
iter=8 Train=0.942841 Test=0.946035
iter=9 Train=0.942563 Test=0.94575
Time:		1497 seconds


In [8]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
   --pairwise 0 --regularization_const 0.001 --learning_method ALS --hash_size -1 -g memory -i 10
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 136726
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 136726
iter=0 Train=0.897143 Test=0.90701
iter=1 Train=0.880088 Test=0.889907
iter=2 Train=0.879318 Test=0.889137
iter=3 Train=0.879249 Test=0.889064
iter=4 Train=0.87923 Test=0.889042
^C
Time:		867 seconds


### Добавим парные взаимодействия

In [7]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
   --pairwise 4 --regularization_const 0.0001 --learning_method ALS --hash_size 64000 -g memory -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 63999
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 63999
iter=0 Train=0.929394 Test=0.943071
iter=1 Train=0.914353 Test=0.930317
iter=2 Train=0.907857 Test=0.924378
iter=3 Train=0.904354 Test=0.921056
iter=4 Train=0.902118 Test=0.918904
Time:		4485 seconds


In [9]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
   --pairwise 4 --regularization_const 0.0001 --learning_method ALS --hash_size -1 -g memory -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 136726
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 136726
iter=0 Train=0.853279 Test=0.895358
iter=1 Train=0.823272 Test=0.872093
iter=2 Train=0.809276 Test=0.860907
iter=3 Train=0.802292 Test=0.855558
iter=4 Train=0.798409 Test=0.85273
Time:		3380 seconds


**В очередной раз ALS с парными фичами оставляет далеко позади остальные алгоритмы**

## Vowpal Wabbit

Приведем данные к vw формату

In [3]:
fm2vw('Generated/train_10m_reg.fm', 'Generated/train_10m_reg.vw')
fm2vw('Generated/test_10m_reg.fm', 'Generated/test_10m_reg.vw')

7500041it [00:06, 1246628.29it/s]
2500013it [00:02, 1111723.69it/s]


In [31]:
t_start = time.time()
!vw Generated/train_10m_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.01 -b 18 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_10m_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 41 seconds
Start evaluation
Validation RMSE:	 1.1104841255494076
Time:		60 seconds


In [37]:
t_start = time.time()
!vw Generated/train_10m_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 18 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_10m_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 41 seconds
Start evaluation
Validation RMSE:	 1.093433365227215
Time:		61 seconds


In [38]:
t_start = time.time()
!vw Generated/train_10m_reg.vw -f VwHelpFiles/model --loss_function squared --sgd --learning_rate 0.001 -b 18 --passes 10 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_10m_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 83 seconds
Start evaluation
Validation RMSE:	 1.0729906582802802
Time:		103 seconds


In [2]:
t_start = time.time()
!vw Generated/train_10m_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.01 -b 18 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_10m_reg', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 48 seconds
Start evaluation
Validation RMSE:	 0.9868549151497806
Time:		71 seconds


In [5]:
t_start = time.time()
!vw Generated/train_10m_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.01 -b 18 --passes 10 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_10m_reg', 'VwHelpFiles/pred')))
!rm VwHelpFiles/pred
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 87 seconds
Start evaluation
Validation RMSE:	 0.9586356933058612
Time:		107 seconds


In [7]:
t_start = time.time()
!vw Generated/train_10m_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.01 -b 18 --passes 80 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation RMSE:\t {}'.format(get_rmse('Generated/ytest_10m_reg', 'VwHelpFiles/pred')))
!rm VwHelpFiles/pred
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 706 seconds
Start evaluation
Validation RMSE:	 0.9089684082685677
Time:		728 seconds


# 10M Classification

### Разделим все фильмы на условно **хорошие**(4-5) и **плохие**(1-3)

In [14]:
reg2fm('Generated/train_10m_reg.fm', 'Generated/train_10m_class.fm')
reg2fm('Generated/test_10m_reg.fm', 'Generated/test_10m_class.fm')

!head -n 3 Generated/train_10m_class.fm

7500041it [00:10, 696628.35it/s]
2500013it [00:03, 647881.75it/s]

0 139:1 71690:1 136702:0.4 136707:1 136722:1 
1 215:1 71690:1 136702:0.7 136707:1 136722:1 
0 217:1 71690:1 136702:0.1 136707:1 136722:1 


In [19]:
!head -n 3 Generated/test_10m_class.fm

1 65480:1 71924:1 136702:0.6 136707:1 136710:1 136722:1 136725:1 
1 35098:1 72656:1 136702:0.4 136710:1 136718:1 136722:1 
1 71509:1 78636:1 136702:1e+00 136710:1 136719:1 136722:1 


### FM SGD

In [11]:
t_start = time.time()
!./FM --bias true --linear true --task_type classification -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
   --learning_rate 0.00000001 --pairwise 0 --regularization_const 0.001 --learning_method SGD --hash_size -1 -g inplace -i 4
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 136726
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 136726
iter=0 Train=4.95061 Test=4.95079
iter=1 Train=2.76833 Test=2.76839
iter=2 Train=1.50514 Test=1.50513
iter=3 Train=0.58781 Test=0.58778
Time:		641 seconds


In [12]:
t_start = time.time()
!./FM --bias true --linear true --task_type classification -t Generated/train_10m_reg.fm -e Generated/test_10m_reg.fm \
   --learning_rate 0.00000001 --pairwise 0 --regularization_const 0.001 --learning_method SGD --hash_size 32000 -g inplace -i 4
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 7500041 rows
Target from 0 to 5
Max feature index is 31999
Preprocessing
Processed 2500013 rows
Target from 0 to 5
Max feature index is 31999
iter=0 Train=4.94939 Test=4.94955
iter=1 Train=2.76714 Test=2.76718
iter=2 Train=1.50393 Test=1.50392
iter=3 Train=0.58656 Test=0.586517
Time:		860 seconds


## VW

In [5]:
reg2vw('Generated/train_10m_reg.fm', 'Generated/train_10m_class.vw')
reg2vw('Generated/test_10m_reg.fm', 'Generated/test_10m_class.vw')

!head -n 3 Generated/train_10m_class.vw

7500041it [00:13, 544168.79it/s]
2500013it [00:06, 398136.44it/s]

-1 | 139:1 71690:1 136702:0.4 136707:1 136722:1 
1 | 215:1 71690:1 136702:0.7 136707:1 136722:1 
-1 | 217:1 71690:1 136702:0.1 136707:1 136722:1 


In [20]:
input_file = open('Generated/ytest_10m_reg',  'r')
out_file   = open('Generated/ytest_10m_class', 'w')
for line in tqdm(input_file):
    target = str((int(line[0]) > 3) * 2 - 1)
    out_file.write(target + '\n')        
!head -n 3 Generated/ytest_10m_class

2500013it [00:02, 867064.31it/s]


1
1
1


In [29]:
t_start = time.time()
!vw Generated/train_10m_class.vw -f VwHelpFiles/model --loss_function logistic --learning_rate 0.001 -b 18 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_class.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation logloss:\t {}'.format(get_log_loss('Generated/ytest_10m_class', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 53 seconds
Start evaluation
Validation logloss:	 0.6331332680570104
Time:		85 seconds


In [24]:
t_start = time.time()
!vw Generated/train_10m_class.vw -f VwHelpFiles/model --loss_function logistic --learning_rate 0.001 -b 18 --passes 20 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_class.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation logloss:\t {}'.format(get_log_loss('Generated/ytest_10m_class', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 215 seconds
Start evaluation
Validation logloss:	 0.6176305147942617
Time:		246 seconds


In [25]:
t_start = time.time()
!vw Generated/train_10m_class.vw -f VwHelpFiles/model --loss_function logistic --learning_rate 0.001 -b 18 --passes 80 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_10m_class.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation logloss:\t {}'.format(get_log_loss('Generated/ytest_10m_class', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 881 seconds
Start evaluation
Validation logloss:	 0.6058598137637439
Time:		927 seconds


# 20M Dataset

In [2]:
folder = 'SourceData/ml-20m/'
test_size = 0.25
train, test = build_movielens(
    folder, test_size, with_genres=True,
    with_users_info=False, with_rated_movies=False)
train.head()

load ratings....
calculation of monthes....
load movies....
build genres ohe....
join dataframes....
users info skipped
train/test split...
rated movies history skipped
preprocessing done....


,user,movie,rating,monthes,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,...,genre_14,genre_15,genre_16,genre_17,genre_18,genre_19,genre_20,genre_21,genre_22,genre_23
0,1,2,7,0.505859,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,29,2,6,0.068420,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,34,2,6,0.087463,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,54,2,6,0.289062,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,91,2,7,0.505859,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
field_info = get_offset_stats(train, test)
train2format(train, fm_extractor, field_info, 'Generated/train_20m_reg.fm',)
test2format(
    test, fm_extractor, field_info,
    'Generated/test_20m_reg.fm',
    'Generated/ytest_20m_reg'
)
!head -n 3 Generated/train_20m_reg.fm

100%|██████████| 5000065/5000065 [24:23<00:00, 3416.39it/s]


7 1:1 138496:1 269757:0.5 269759:1 269761:1 269766:1 
6 29:1 138496:1 269757:0.07 269759:1 269761:1 269766:1 
6 34:1 138496:1 269757:0.09 269759:1 269761:1 269766:1 


In [3]:
# make targets from 1 to 5.5
target_transform('Generated/train_20m_reg.fm', 'Generated/train_20m_reg_2.fm')
target_transform('Generated/test_20m_reg.fm', 'Generated/test_20m_reg_2.fm')
target_transform('Generated/ytest_20m_reg', 'Generated/ytest_20m_reg_2')

15000198it [00:30, 486734.70it/s]
5000065it [00:11, 438827.64it/s]
5000065it [00:07, 696166.74it/s]


In [25]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 0 --task_type regression -t Generated/train_20m_reg_2.fm -e Generated/test_20m_reg_2.fm \
    --learning_rate 0.003 --regularization_const 0.01 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 15000198 rows
Target from 1 to 5
Max feature index is 269781
Preprocessing
Processed 5000065 rows
Target from 1 to 5
Max feature index is 269781
iter=0 Train=1.04622 Test=1.04807
iter=1 Train=1.00683 Test=1.00975
iter=2 Train=1.00094 Test=1.00458
iter=3 Train=1.00305 Test=1.00721
iter=4 Train=1.00718 Test=1.01176
Time:		1459 seconds


In [28]:
t_start = time.time()
!./FM --bias true --linear true --pairwise 4 --task_type regression -t Generated/train_20m_reg_2.fm -e Generated/test_20m_reg_2.fm \
    --learning_rate 0.01 --regularization_const 0.001 --learning_method SGD --hash_size -1 -g inplace -i 5
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 15000198 rows
Target from 1 to 5
Max feature index is 269781
Preprocessing
Processed 5000065 rows
Target from 1 to 5
Max feature index is 269781
iter=0 Train=1.00771 Test=1.01195
iter=1 Train=0.973426 Test=0.980399
iter=2 Train=0.968957 Test=0.978308
iter=3 Train=0.977346 Test=0.988578
iter=4 Train=0.99194 Test=1.00459
Time:		2324 seconds


In [2]:
t_start = time.time()
!./FM --bias true --linear true --task_type regression -t Generated/train_20m_reg_2.fm -e Generated/test_20m_reg_2.fm \
   --pairwise 0 --regularization_const 0.0001 --learning_method ALS --hash_size 64000 -g memory -i 10
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Preprocessing
Processed 15000198 rows
Target from 1 to 5
Max feature index is 63999
Preprocessing
iter=0 Train=0.935272 Test=0.937465
Time:		17246 seconds


## VW

In [11]:
fm2vw('Generated/train_20m_reg_2.fm', 'Generated/train_20m_reg.vw', split_pos=3)
fm2vw('Generated/test_20m_reg_2.fm', 'Generated/test_20m_reg.vw', split_pos=3)

!head -n 3 Generated/train_20m_reg.vw

15000198it [00:16, 906640.28it/s]
5000065it [00:05, 921427.60it/s]


4.0 |  1:1 138496:1 269757:0.5 269759:1 269761:1 269766:1 
3.5 |  29:1 138496:1 269757:0.07 269759:1 269761:1 269766:1 
3.5 |  34:1 138496:1 269757:0.09 269759:1 269761:1 269766:1 


In [12]:
t_start = time.time()
!vw Generated/train_20m_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.1 -b 18 --passes 5 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_20m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation logloss:\t {}'.format(get_rmse('Generated/ytest_20m_reg_2', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 95 seconds
Start evaluation
Validation logloss:	 0.8990432981602857
Time:		147 seconds


In [13]:
t_start = time.time()
!vw Generated/train_20m_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.1 -b 18 --passes 10 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_20m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation logloss:\t {}'.format(get_rmse('Generated/ytest_20m_reg_2', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 182 seconds
Start evaluation
Validation logloss:	 0.8799434325077914
Time:		227 seconds


In [14]:
t_start = time.time()
!vw Generated/train_20m_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.1 -b 18 --passes 40 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_20m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation logloss:\t {}'.format(get_rmse('Generated/ytest_20m_reg_2', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 684 seconds
Start evaluation
Validation logloss:	 0.8621596496482833
Time:		728 seconds


In [15]:
t_start = time.time()
!vw Generated/train_20m_reg.vw -f VwHelpFiles/model --loss_function squared --learning_rate 0.1 -b 18 --passes 80 --cache_file VwHelpFiles/cache --quiet
print('Training done - {0} seconds'.format(int(time.time() - t_start)))
! vw -i VwHelpFiles/model -t Generated/test_20m_reg.vw -r VwHelpFiles/pred --quiet
!rm VwHelpFiles/model
!rm VwHelpFiles/cache
print('Start evaluation')
print('Validation logloss:\t {}'.format(get_rmse('Generated/ytest_20m_reg_2', 'VwHelpFiles/pred')))
print('Time:\t\t{} seconds'.format(int(time.time() - t_start)))

Training done - 883 seconds
Start evaluation
Validation logloss:	 0.8611600615122224
Time:		932 seconds
